In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [19]:
# state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/Arkansas_2018.csv")

# no state or zip code
no_state_no_zip = ['Louisiana', 'Maine', 'South_Dakota', 'Virginia']

# no city or state or zip code
no_city_no_state = ['Minnesota', 'Vermont',]

# no commas
no_commas = ['Arkansas', 'Kentucky', 'Mississippi', 'New_Hampshire', 'New_Jersey']

# already clean
cleaned = ['Alaska', 'Connecticut', 'Delaware', 'Georgia', 'Iowa', 'Illinois', 'Maryland', 'Oklahoma', 'Montana', 'Nebraska', 'North_Carolina',
'Pennsylvania', 'Rhode_Island', 'South_Carolina', 'West_Virginia', 'Wisconsin', "New_Mexico", 'North_Dakota', 'Ohio']

len(cleaned)

19

In [20]:
mass = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/Massachusetts_2018.csv")
def isMA(output):

    '''
    First the first occuracne of Vote and the last occurance of . Remove all text in between that position
    Remove Words before - in the begining 
    e.g. SCHOOL - 165 
    '''

    for i,r in output.iterrows():
        if output.loc[i,'state']=='MA':
            if "vote" in output.loc[i,'address'].lower() or "voter" in output.loc[i,'address'].lower():
                start_pos=output.loc[i,'address'].lower().find("vote")
                end_pos=output.loc[i,'address'].rfind(".")
                address=output.loc[i,'address']

                if end_pos==len(output.loc[i,'address'])-1:

                    address=output.loc[i,'address'][:start_pos]+""+"MA"

                else:
                    address=output.loc[i,'address'][:start_pos]+" "+output.loc[i,'address'][end_pos+2:]

                address=address.replace(".",",")

                if address[-2:]!='MA':
                    address=address+' MA'
                output.loc[i,'address']=address


            if "-" in output.loc[i,'address'].lower():
                address=output.loc[i,'address']

                end_pos=output.loc[i,'address'].lower().find("-")
                address=address[end_pos+1:]

                output.loc[i,'address']=address
    output.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/Massachusetts_2018_clean.csv") 

    return output

new_mass = isMA(mass)
new_mass

,state,jurisdiction_type,county_name,jurisdiction,precinct_id,name,address
0,MA,town,Plymouth,ABINGTON,0-01,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
1,MA,town,Plymouth,ABINGTON,0-02,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
2,MA,town,Plymouth,ABINGTON,0-03,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
3,MA,town,Plymouth,ABINGTON,0-04,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
4,MA,town,Plymouth,ABINGTON,0-05,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
...,...,...,...,...,...,...,...
2169,MA,town,Barnstable,YARMOUTH,0-3,TOWN HALL HEARING ROOM,"1146 RTE.28,S.YARMOUTH"
2170,MA,town,Barnstable,YARMOUTH,0-4,YARMOUTH SENIOR CENTER,"528 FOREST ROAD, SOUTH YARMOUTH"
2171,MA,town,Barnstable,YARMOUTH,0-5,CC&ISLANDS ASSOC REALTORS,"22 MID TECH DRIVE, WEST YARMOUTH"
2172,MA,town,Barnstable,YARMOUTH,0-6,CC&ISLANDS ASSOC REALTORS,"22 MID TECH DRIVE, WEST YARMOUTH"


In [21]:
def add_state(states):
    for state in states:
        #Load state CSV
        state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/{state}_2018.csv")
        state_df.columns = [x.lower() for x in state_df.columns]
        #Remove NA's
        state_df = state_df[state_df['address'].notna()]
        #Remove Duplicates
        state_df = state_df.drop_duplicates(subset=['name','address'], ignore_index=True)
        state_df['address'] = state_df['address'].astype(str) + ", " + state_df["state"]
        state_df.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv") 

    # return state_df
add_state(no_state_no_zip)

In [23]:
def add_city_state(states):
        for state in states:
            #Load state CSV
            state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/{state}_2018.csv")
            #Remove NA's
            state_df = state_df[state_df['address'].notna()]
            #Remove Duplicates
            state_df = state_df.drop_duplicates(subset=['name','address'], ignore_index=True)
            state_df.columns = [x.lower() for x in state_df.columns]
            
            if state == 'Virginia':
                #add city and state with commas 
                state_df['address'] = state_df['address'].astype(str) + ", " + state_df['name'].astype(str) + ', ' + state_df['state'].astype(str)
            else: 
                #add city and state with commas
                state_df['address'] = state_df['address'].astype(str) + ", " + state_df['precinct_name'].astype(str) + ', '+ state_df['state'].astype(str)

            state_df.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv") 
        # return state_df
add_city_state(no_city_no_state)

In [24]:
def already_cleaned(states):
        for state in states:
            #Load state CSV
            state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/{state}_2018.csv")
            state_df.columns = [x.lower() for x in state_df.columns]
            #Remove NA's
            state_df = state_df[state_df['address'].notna()]
            #Remove Duplicates
            state_df = state_df.drop_duplicates(subset=['name','address'], ignore_index=True) 
            state_df.columns = [x.lower() for x in state_df.columns]
            state_df.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv") 
already_cleaned(cleaned)

In [25]:

def add_commas(states):
        for state in states:
            #Load state CSV
            state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/{state}_2018.csv")
            state_df.columns = [x.lower() for x in state_df.columns]
            #Remove NA's
            state_df = state_df[state_df['address'].notna()]
            #Remove Duplicates
            state_df = state_df.drop_duplicates(subset=['name','address'], ignore_index=True)
            if state == ('New_Hamshire' or 'Mississippi'):
                if isinstance(address[0],str):
                    m = re.search(r"\d", address)
                    address=address[m.start():]

            for i in range(0,len(state_df)):
                #Add street, state, and zip code commas
                start_list = state_df.loc[i,'address'].split()
                #Catch Arkansas addresses that are short
                if (len(start_list) > 3):
                    draft_str2 = ' '.join([str(start_list[i]) for i in range(0,len(start_list) - 3)])
                    new_str = ', '.join([draft_str2, start_list[-3], start_list[-2], start_list[-1]])
                else:
                    new_str = ', '.join(start_list)
                state_df.loc[i,'address'] = new_str
            state_df.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv")

add_commas(no_commas)

In [37]:
import glob
data = pd.DataFrame()
for filename in glob.glob(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/*"):
    df = pd.read_csv(filename)
    data = pd.concat([data, df['address']], ignore_index= True)

# pd.read_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv")

# Misc Analysis For Vaughan

In [41]:
data.columns = ['address']
data
data.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/cleaned_2018_polling_place.csv")

In [42]:
df = pd.read_csv('/Users/dapoadegbile/Downloads/cleaned_2018_polling_place.csv')

In [43]:
df['Accuracy Type'].value_counts(normalize = True)

rooftop                  0.607485
range_interpolation      0.183954
place                    0.117116
street_center            0.041634
nearest_rooftop_match    0.026955
intersection             0.016058
state                    0.005905
county                   0.000894
Name: Accuracy Type, dtype: float64

In [56]:
df = pd.read_csv('/Users/dapoadegbile/Documents/Github/Capstone /mtv_viacom_capstone/20_intermediate_files/30_campuses_w_dist_to_nearest_pp.csv', header = 1)
# df1 = pd.read_json('/Users/dapoadegbile/Documents/Github/Capstone /mtv_viacom_capstone/20_intermediate_files/30_campuses_w_dist_to_nearest_pp.geojson')

In [57]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0_x,UNIQUEID,NAME,ADDRESS,CITY_x,STATE,ZIP_x,...,2020_early_distance_by_driving (miles),2020_early_duration_by_driving (minutes),2020_early_distance_by_transit (miles),2020_early_duration_by_transit (minutes),2020_distance_by_walking (miles),2020_duration_by_walking (minutes),2020_distance_by_driving (miles),2020_duration_by_driving (minutes),2020_distance_by_transit (miles),2020_duration_by_transit (minutes)
0,0,0.0,0.0,0.0,212878.0,HARRISBURG AREA COMMUNITY COLLEGE,1 HACC DR.,Harrisburg,PA,17110.0,...,3.976774,12.0,4.101049,22.0,3.603952,71.0,4.038912,10.0,NaN,NaN
1,1,1.0,1.0,1.0,214111.0,MONTGOMERY COUNTY COMMUNITY COLLEGE,340 DEKALB PIKE,Blue Bell,PA,19422.0,...,6.400121,17.0,10.128347,42.0,10.563307,1.0,1.242742,1.0,10.563307,1.0
2,2,2.0,2.0,2.0,213376.0,LACKAWANNA COLLEGE,501 VINE ST,Scranton,PA,18509.0,...,0.497097,3.0,0.497097,10.0,0.062137,1.0,0.248548,2.0,0.062137,1.0
3,3,3.0,3.0,3.0,215655.0,ROBERT MORRIS UNIVERSITY,6001 UNIVERSITY BOULEVARD,Moon Township,PA,15108.0,...,11.246815,19.0,21.747985,126.0,2.423347,50.0,3.355403,8.0,2.423347,50.0
4,4,4.0,4.0,4.0,212869.0,HARCUM COLLEGE,750 MONTGOMERY AVE,Bryn Mawr,PA,19010.0,...,3.355403,9.0,4.101049,26.0,0.248548,5.0,0.248548,1.0,0.248548,5.0


In [62]:
new_states = df[df['STATE'].isin(['MI' , 'WI' , 'FL' , 'GA' , 'TX' , 'NC' , 'AZ'])]
new_states.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0_x,UNIQUEID,NAME,ADDRESS,CITY_x,STATE,ZIP_x,...,2020_early_distance_by_driving (miles),2020_early_duration_by_driving (minutes),2020_early_distance_by_transit (miles),2020_early_duration_by_transit (minutes),2020_distance_by_walking (miles),2020_duration_by_walking (minutes),2020_distance_by_driving (miles),2020_duration_by_driving (minutes),2020_distance_by_transit (miles),2020_duration_by_transit (minutes)
239,239,239.0,239.0,239.0,199193.0,NORTH CAROLINA STATE UNIVERSITY AT RALEIGH,2101 HILLSBOROUGH STREET,Raleigh,NC,27695.0,...,1.242742,5.0,2.547621,21.0,2.236936,44.0,2.547621,6.0,3.168992,26.0
240,240,240.0,240.0,240.0,198039.0,BLUE RIDGE COMMUNITY COLLEGE,180 WEST CAMPUS DRIVE,Flat Rock,NC,28731.0,...,2.112661,6.0,2.050524,41.0,1.615565,32.0,1.739839,5.0,1.615565,32.0
241,241,241.0,241.0,241.0,197814.0,COLLEGE OF THE ALBEMARLE,1208 N ROAD ST,Elizabeth City,NC,27906.0,...,5.778750,12.0,NaN,NaN,1.304879,26.0,1.304879,4.0,1.304879,26.0
242,242,242.0,242.0,242.0,197850.0,SOUTH PIEDMONT COMMUNITY COLLEGE,680 HIGHWAY 74 WEST,Polkton,NC,28135.0,...,9.196291,14.0,NaN,NaN,0.994194,20.0,1.180605,4.0,0.994194,20.0
243,243,243.0,243.0,243.0,197869.0,APPALACHIAN STATE UNIVERSITY,287 RIVERS ST,Boone,NC,28608.0,...,1.118468,6.0,0.869919,18.0,0.869919,18.0,1.118468,6.0,0.869919,18.0


In [95]:
new_states[new_states['distances_2020_early'] == 0][['NAME', 'STATE']]

,NAME,STATE
239,NORTH CAROLINA STATE UNIVERSITY AT RALEIGH,NC
243,APPALACHIAN STATE UNIVERSITY,NC
254,CAPE FEAR COMMUNITY COLLEGE,NC
260,CLEVELAND COMMUNITY COLLEGE,NC
265,DUKE UNIVERSITY,NC
266,DURHAM TECHNICAL COMMUNITY COLLEGE,NC
267,EAST CAROLINA UNIVERSITY,NC
269,ELIZABETH CITY STATE UNIVERSITY,NC
301,NORTH CAROLINA A &amp; T STATE UNIVERSITY,NC
302,UNIVERSITY OF NORTH CAROLINA AT ASHEVILLE,NC


In [76]:
# new_states[new_states['2020_early_distance_by_driving (miles)']].isnull()

In [84]:
(df[df['2020_early_distance_by_driving (miles)'].isnull()])

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0_x,UNIQUEID,NAME,ADDRESS,CITY_x,STATE,ZIP_x,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,preprocessed_name,centroid_x_epsg_102005,centroid_y_epsg_102005,centroid_long,centroid_lat,OPE ID,IPED ID,School Name,Institution Type,# of Nat'l AP Partners,# of Partners,Civic Holiday,MSI,NSLVE,NSLVE 2014,2016 NSLVE Report,2018 NSLVE Reports,2016 Action Plan,2018 Action Plan,2020 Action Plan,Campus Contact,Longitude,Latitude,Pell Grant %,BIPOC %,American Indian or Alaskan Native,Asian,Black or African American,Hispanic or Latinx,NHPI,Two or more Races,geometry,index_right,state_2020,Latitude_2020,Longitude_2020,distances_2020,Latitude_2020_early,Longitude_2020_early,distances_2020_early,Latitude_starbucks,Longitude_starbucks,distances_starbucks,Latitude_target,Longitude_target,distances_target,Unnamed: 0_y,UNITID,INSTNM,IALIAS,ADDR,CITY_y,STABBR,ZIP_y,FIPS,OBEREG,CHFNM,CHFTITLE,GENTELE,EIN,DUNS,OPEID,OPEFLAG,WEBADDR,ADMINURL,FAIDURL,APPLURL,NPRICURL,VETURL,ATHURL,DISAURL,SECTOR,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,HDEGOFR1,DEGGRANT,HBCU,HOSPITAL,MEDICAL,TRIBAL,LOCALE,OPENPUBL,ACT,NEWID,DEATHYR,CLOSEDAT,CYACTIVE,POSTSEC,PSEFLAG,PSET4FLG,RPTMTH,INSTCAT,C18BASIC,C18IPUG,C18IPGRD,C18UGPRF,C18ENPRF,C18SZSET,C15BASIC,CCBASIC,CARNEGIE,LANDGRNT,INSTSIZE,F1SYSTYP,F1SYSNAM,F1SYSCOD,CBSA,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG,EFBAGE,LINE,LSTUDY,EFAGE01,EFAGE02,EFAGE03,EFAGE04,EFAGE05,EFAGE06,EFAGE07,EFAGE08,EFAGE09,Total_Enrollment,2020_early_distance_by_walking (miles),2020_early_duration_by_walking (minutes),2020_early_distance_by_driving (miles),2020_early_duration_by_driving (minutes),2020_early_distance_by_transit (miles),2020_early_duration_by_transit (minutes),2020_distance_by_walking (miles),2020_duration_by_walking (minutes),2020_distance_by_driving (miles),2020_duration_by_driving (minutes),2020_distance_by_transit (miles),2020_duration_by_transit (minutes)
930,930,930.0,930.0,930.0,196291.0,SUNY MARITIME COLLEGE,6 PENNYFIELD AVENUE,Throggs Neck,NY,10465.0,4198,(718) 409-7200,1.0,A,2093.0,BRONX,36005.0,USA,611310.0,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",http://maritime.prod.acquia-sites.com/sites/de...,2018-09-19,IMAGERY/OTHER,2018-09-19,www.sunymaritime.edu,1734.0,359.0,NOT AVAILABLE,sunymaritimecollege,1.845844e+06,426405.25497,-73.795033,40.806636,2853.00,196291.00,SUNY Maritime College at Fort Schulyler,"Public, 4 Year",0.0,0.0,0.0,,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-73.795309,40.807217,22,23.82%,0.05%,4.50%,5.11%,11.81%,0.00%,2.35%,"POLYGON ((1845214.843451653 426542.3919438811,...",NaN,NaN,NaN,NaN,NaN,40.815817,-73.814405,1275.169039,40.78032,-73.776752,2917.25512,40.824145,-73.837659,3427.588497,1300.0,196291.0,SUNY Maritime College,,6 Pennyfield Avenue,Throggs Neck,NY,10465-4198,36.0,2.0,RADM Michael Alfultis,President,7.184097e+09,146013200.0,142359731,285300.0,1.0,www.sunymaritime.edu/,www.sunymaritime.edu/Admissions,www.sunymaritime.edu/cost-aid-0,www.sunymaritime.edu/admissions/apply-now,www.suny.edu/howmuch/netpricecalculator.xhtml?...,www.sunymaritime.edu/admissions/veteran-inform...,,www.sunymaritime.edu/student-life/academic-sup...,1.0,1.0,1.0,7.0,1.0,1.0,20.0,1.0,2.0,2.0,2.0,2.0,11.0,1.0,A,-2.0,-2.0,-2.0,1.0,1.0,1.0,1.0,1.0,2.0,20.0,18.0,7.0,14.0,3.0,11.0,20.0,22.0,59.0,2.0,2.0,1.0,State University of New York System ...,136040.0,35620.0,1.0,408.0,-2.0,36005.0,Bronx County,3614.0,-73.795309,40.807217,124.0,1.0,1.0,412.0,1.0,1312.0,218.0,123.0,18.0,1530.0,141.0,1435.0,236.0,1671.0,1671.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2254,=======,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [91]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [83]:
df.columns 
dir(a)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__inv__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_accessors',
 '_arith_method',
 '_assert_can_do_setop',